In [1]:
import numpy as np
import pandas as pd
import prettytable

In [2]:
import time
main_start=time.time()
time_log={"main_start":main_start}


In [3]:
mpstDF= pd.read_csv("mpst.csv")
mpstDF

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [4]:
final_mpstDF_train=mpstDF[mpstDF["split"]=="train"]
final_mpstDF_validation=mpstDF[mpstDF["split"]=="val"]
final_mpstDF_test=mpstDF[mpstDF["split"]=="test"]
print("Shape Train",final_mpstDF_train.shape)
print("Shape Validation",final_mpstDF_validation.shape)
print("Shape Test",final_mpstDF_test.shape)

Shape Train (9489, 6)
Shape Validation (2373, 6)
Shape Test (2966, 6)


In [5]:
final_mpstDF_train.index

Int64Index([    0,     1,     3,     6,     7,     8,     9,    10,    11,
               12,
            ...
            14810, 14811, 14813, 14815, 14817, 14818, 14820, 14822, 14824,
            14825],
           dtype='int64', length=9489)

In [6]:
final_mpstDF_validation.index

Int64Index([    4,     5,    18,    20,    26,    32,    40,    48,    53,
               65,
            ...
            14767, 14769, 14772, 14777, 14779, 14789, 14795, 14796, 14812,
            14821],
           dtype='int64', length=2373)

In [7]:
final_mpstDF_test.index

Int64Index([    2,    15,    19,    24,    27,    31,    35,    37,    41,
               42,
            ...
            14776, 14801, 14806, 14808, 14814, 14816, 14819, 14823, 14826,
            14827],
           dtype='int64', length=2966)

In [8]:
#READING THE BERT EMBEDDINGS AND Y MATRIX
xl_embedding=np.load(r"D:\CodeRepo\Thesis\Thesis\XLNet\xl_embeddings.npz")
label_values=np.load(r"D:\CodeRepo\Thesis\Thesis\EDA\Y.npz")

In [9]:
type1_XL_Embeddings=xl_embedding["t1"]
type2_XL_Embeddings=xl_embedding["t2"]
label_values=label_values["arr_0"]

In [10]:
type1_XL_Embeddings.shape

(14828, 768)

In [11]:
label_values.shape

(14828, 71)

In [13]:
def get_partition_Embeddings(x_t1,x_t2,y,df,partition_nm):
    _df=df[df["split"]==partition_nm]
    index_list=list(_df.index)
    temp_array_x_t1=[]
    temp_array_x_t2=[]
    temp_array_y=[]
    for index in index_list:
        temp_array_x_t1.append(x_t1[index,:])
        temp_array_x_t2.append(x_t2[index,:])
        temp_array_y.append(y[index,:])
    temp_array_x_t1=np.array(temp_array_x_t1)
    temp_array_x_t2=np.array(temp_array_x_t2)
    temp_array_y=np.array(temp_array_y)
    return temp_array_x_t1,temp_array_x_t2, temp_array_y

In [14]:
# FOR TRAIN
type1_XL_Embeddings_Train,type2_XL_Embeddings_Train,label_values_Train=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"train")
# FOR VALIDATION
type1_XL_Embeddings_Val,type2_XL_Embeddings_Val,label_values_Val=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"val")
# FOR TEST
type1_XL_Embeddings_Test,type2_XL_Embeddings_Test,label_values_Test=get_partition_Embeddings(type1_XL_Embeddings,type2_XL_Embeddings,label_values,mpstDF,"test")

In [16]:
# SEPARTAING THE TRAIN TEST AND VALIDATION ROWS
print(type1_XL_Embeddings_Train.shape)
print(type2_XL_Embeddings_Train.shape)
print(type1_XL_Embeddings_Val.shape)
print(type2_XL_Embeddings_Val.shape)
print(type1_XL_Embeddings_Test.shape)
print(type2_XL_Embeddings_Test.shape)
print(label_values_Train.shape)
print(label_values_Val.shape)
print(label_values_Test.shape)


(9489, 768)
(9489, 768)
(2373, 768)
(2373, 768)
(2966, 768)
(2966, 768)
(9489, 71)
(2373, 71)
(2966, 71)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix, hamming_loss,classification_report

# Activation Functions

# def _identity(x):
#     return x
# def _binary_step(x, threshold = 0):
#     return 1 if x=threshold else 0
# def _biploar_step(x ,threshold = 0):
#     return 1 if x>=threshold else -1
# def _binary_sigmoid(x):
#     return 1. / (1. + np.exp(-x))
# def _bipolar_sigmoid(x):
#     return (1. - np.exp(-x))/(1. + np.exp(-x))
# def _relu_function(x):
#     return np.max(0, x)
# def _relu_leaky(x):
#     return np.max(0.01*x, x)


_identity =np.vectorize(lambda x: x)
_binary_step =np.vectorize(lambda x,t=0: 1 if x>t else 0)
_biploar_step =np.vectorize(lambda x,t=0: 1 if x>t else -1)
_binary_sigmoid=np.vectorize(lambda x: 1. / (1. + np.exp(-x)))
_bipolar_sigmoid=np.vectorize(lambda x: (1. - np.exp(-x))/(1. + np.exp(-x)))
_relu_function=np.vectorize(lambda x: np.max([0, x]))
_relu_leaky=np.vectorize(lambda x: np.max([0.01*x, x]))

In [ ]:


class ELM_MultiLabel:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, activation, bias=True, random_gen="uniform"):
        """
        Args:
            input_nodes ([integer]): Number of Input nodes
            hidden_nodes ([integer]): Number of hidden nodes
            output_nodes ([integer]): Number of output nodes
            activation ([function]): The function which will be used as the activation function in the hidden layer
            bias ([boolean]): Flag to use bias, if True then randomly generate bias @random_gen else bias - 0.
            random_gen (str, optional): The type way in which random weight are generated. Defaults to "uniform".
        """
        self.__input_nodes = input_nodes
        self.__hidden_nodes = hidden_nodes
        self.__output_nodes = output_nodes
        if random_gen == "uniform":
            self.__beta = np.random.uniform(-1.,1.,size = (self.__hidden_nodes, self.__output_nodes))
            self.__alpha = np.random.uniform(-1.,1.,size = (self.__input_nodes, self.__hidden_nodes))
            self.__bias = np.random.uniform(size = (self.__hidden_nodes,))
        else:
            self.__beta = np.random.normal(-1.,1.,size=(self.__hidden_nodes, self.__output_nodes))
            self.__alpha = np.random.normal(-1.,1.,size=(self.__input_nodes, self.__hidden_nodes))
            self.__bias = np.random.normal(size=(self.__n_hidden_nodes,)) 
        self.__activation = activation # Sigmoid Function

    def getInputNodes(self):
        return  self.__input_nodes

    def getHiddenNodes(self):
        return  self.__hidden_nodes

    def getOutputNodes(self):
        return  self.__output_nodes
    
    def getBetaWeights(self):
        return self.__beta
    
    def getAlphaWeight(self):
        return self.__alphs
    
    def getBias(self):
        return self.__bias

    def __get_H_matrix(self, train_x, verbose=False):
        # 1 Propagate data from Input to hidden Layer
        if verbose:
            print("Propagate data from Input to hidden Layer")
        inp = np.dot(train_x , self.__alpha)
        if verbose:
            print(inp)
            print("Adding Biases")
        inp = inp  + self.__bias
        if verbose:
            print(inp)
            print("Applyin activation function")
        inp_activation = np.apply_along_axis(self.__activation, 1, inp)
        return inp_activation

    def fit(self, train_x, train_y, verbose = False, show_metrics = True):
        """
        This function calculates the Beta weights or the output weights
        train_x : input matrix
        train_y : output matrix to be predicted or learned upon unipolar

        returns: if test_y is not given then
                returns the predicted output
                if test_y is given then returns predicted output and evaluation metrics dict 
        """
        if verbose:
            print("train_x shape:", train_x.shape)
            print("train_y shape:", train_y.shape)
        inp_activation = self.__get_H_matrix(train_x, verbose)
        # This is the H matrix getting its Moore Penrose Inverse
        if verbose:
            print(inp_activation)
            print("Getting the Generalized Moore Penrose Inverse")
        generalizedInverse = np.linalg.pinv(inp_activation)
        if verbose:
            print(generalizedInverse)
            print("Finding Beta, output weights")
        # Now find output weight matrix Beta 
        # convert input Y values according to the threshold using biploar step function
        predicted_bipolar=  np.apply_along_axis(_biploar_step, 1, train_y)
        self.__beta = np.dot(generalizedInverse, predicted_bipolar)
        if verbose:
            print("Beta Matrix Weights")
            print(self.__beta)

        print("Model Metrics, for Training :")
        return self.predict(train_x, train_y,verbose,show_metrics)
    
    def predict(self, test_x, test_y = None, verbose = False, show_metrics= True):
        """
        preditcts the output for the input test data
        call this after calling the fit.
        test_data shape should be (batch_size,768 or input_nodes)
        output_shape will be (batch_size, 71 or output_nodes)

        returns: if test_y is not given then
                returns the predicted output
                if test_y is given then returns predicted output and evaluation metrics dict
        """
        if verbose:
            print("Predicting outputs")
        inp_activation = self.__get_H_matrix(test_x, verbose)
        output_predicted = np.dot(inp_activation, self.__beta)
        # convert predicted according to the threshold using biploar step function
        predicted_bipolar =  np.apply_along_axis(_biploar_step, 1, output_predicted)
        predicted_binary = np.apply_along_axis(_binary_step, 1, predicted_bipolar)

        if verbose:
            print("predicted output")
            print(output_predicted)
            print("predicted_bipolar")
            print(predicted_bipolar)
            print("predicted_binary")
            print(predicted_binary)
            print("Original Binary")
            print(test_y)

        eval_dict={}
        if (test_y is not None):
            eval_dict=self.__evaluate(test_y,predicted_binary, for_test=False)
        if(test_y is not None):
            return predicted_binary, eval_dict
        else:
            return predicted_binary

    def __evaluate(self, real, predicted, for_test=True):
        """
        real values as 0,1
        predicted values as 0,1
        """
        # Now we find accuracy, precision, recall, Hamming Loss and F1 Measure
        accuracy = accuracy_score(real, predicted)
        hamLoss = hamming_loss(real, predicted)
        # element wise correctness
        term_wise_accuracy=np.sum(np.logical_not(np.logical_xor(real, predicted)))/real.size

        macro_precision = precision_score(real, predicted, average='macro')
        macro_recall = recall_score(real, predicted, average='macro')
        macro_f1 = f1_score(real, predicted, average='macro')

        micro_precision = precision_score(real, predicted, average='micro')
        micro_recall = recall_score(real, predicted, average='micro')
        micro_f1 = f1_score(real, predicted, average='micro')
        
        metricTable=prettytable.PrettyTable()
        metricTable.field_names = ["Metric", "Macro Value", "Micro Value"]
        metricTable.add_row(["Hamming Loss","{0:.3f}".format(hamLoss) ,""])
        metricTable.add_row(["Term Wise Accuracy","{0:.3f}".format(term_wise_accuracy) ,""])

        metricTable.add_row(["Accuracy","{0:.3f}".format(accuracy),""])
        metricTable.add_row(["Precision","{0:.3f}".format(macro_precision),"{0:.3f}".format(micro_precision)])
        metricTable.add_row(["Recall","{0:.3f}".format(macro_recall),"{0:.3f}".format(micro_recall)])
        metricTable.add_row(["F1-measure","{0:.3f}".format(macro_f1),"{0:.3f}".format(micro_f1)])

        print(metricTable)

        print("Metrics @ Literature")
        lit_HamminLosss, lit_accuracy, lit_precision, lit_recall, lit_f1 = self.get_eval_metrics(real,predicted)

        return_dict = {"HiddenNodes": self.getHiddenNodes(),
                "lit_HamminLosss": lit_HamminLosss,
                "lit_accuracy": lit_accuracy,
                "lit_precision": lit_precision,
                "lit_recall": lit_recall,
                "lit_f1": lit_f1,
                "sklearn_hamLoss": lit_accuracy,
                "sklearn_accuracy": accuracy,
                "sklearn_macro_precision": macro_precision,
                "sklearn_micro_precision-": lit_accuracy,
                "sklearn_macro_recall": macro_recall,
                "sklearn_micro_precision": micro_precision,
                "sklearn_macro_f1": macro_f1,
                "sklearn_micro_f1": micro_f1,
                "term_wise_accuracy": term_wise_accuracy,
                }


        print("Test Classification Report")
        print(classification_report(real,predicted))

        return return_dict

    def get_eval_metrics(self, real, predicted, verbose= False):
        err_cnt_accuracy=0
        err_cnt_precision=0
        err_cnt_recall=0
        if verbose:
            print(real)
            print(predicted)
        for x in range(real.shape[0]):
            err_and= np.logical_and(real[x],predicted[x])
            err_or = np.logical_or(real[x],predicted[x])
            # Accuracy
            err_cnt_accuracy +=(sum(err_and)/sum(err_or))

            # Precision
            if sum(err_and) != 0:
                err_cnt_precision += (sum(err_and) / sum(predicted[x]))
            # Recall
            err_cnt_recall += (sum(err_and) / sum(real[x]))
            if verbose:
                print("Iteration :",x)
                print((sum(err_and)/sum(err_or)))
                print(err_and)
                print(err_or)
        
        err_count_hamming = np.zeros((real.shape))

        for i in range(real.shape[0]):
            for j in range(real.shape[1]):
                if real[i,j] != predicted[i,j]:
                    err_count_hamming[1,j] = err_count_hamming[1,j]+1

        sum_err = np.sum(err_count_hamming);
        HammingLoss = sum_err/real.size;
        accuracy = err_cnt_accuracy / real.shape[0]
        precision = err_cnt_precision / real.shape[0]
        recall = err_cnt_recall / real.shape[0]
        f1 = 2*((precision*recall)/(precision+recall))
        if verbose:
            print("Final: ")
            print("Hamming Loss: ", HammingLoss)
            print("Accuracy: ",accuracy)
            print("precision: ",precision)
            print("recall: ",recall)
            print("f1: ",f1)

        metricTable=prettytable.PrettyTable()
        metricTable.field_names = ["Metric", "Value"]
        metricTable.add_row([" Literature Hamming Loss","{0:.3f}".format(HammingLoss)])
        metricTable.add_row(["Literature Accuracy","{0:.3f}".format(accuracy)])

        metricTable.add_row(["Literature Precision","{0:.3f}".format(precision)])
        metricTable.add_row(["LiteratureRecall","{0:.3f}".format(recall)])
        metricTable.add_row(["LiteratureF1-measure","{0:.3f}".format(f1)])

        print(metricTable)

        return HammingLoss,accuracy,precision,recall,f1


Now The preprocessing and is done. We will save the time

In [ ]:
# Creating a object to store the mertics for each model created.

In [ ]:
time_log["End preprocessing"]=time.time()

In [ ]:
# doing a test for ELM model with 100 nodes
#TRAIN
print("TRAINING \n")
t1_emlMLTC_50= ELM_MultiLabel(input_nodes=768, hidden_nodes=50, output_nodes=71, activation=_bipolar_sigmoid)
print("Training Model with 100 hidden nodes")
t1_emlMLTC_50.fit(type1_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)

#TEST
print("TESTING \n")
t1_emlMLTC_50.predict(type1_BERT_Embeddings_Test,label_values_Test, show_metrics=True)


Now We will run the model for all the three types data sets we have viz.
- TRAIN X
  - type1_BERT_Embeddings_Train
  - type2_BERT_Embeddings_Train
- VALIDATION X
  - type1_BERT_Embeddings_Val
  - type2_BERT_Embeddings_Val
- TEST X
  - type1_BERT_Embeddings_Test
  - type2_BERT_Embeddings_Test
- TRAIN Y
  - label_values_Train.shape
- VALIDATION Y
  - label_values_Val.shape
- TEST Y
  - label_values_Test.shape

In [ ]:
list_of_models_hidden_nodes=[100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000, 10000, 15000, 20000]
t1_models={}
t2_models={}

In [ ]:
metrics_dict_list=[]

def add_data_to_metric_list(eval_dict, activation, type, start, phase, end, metrics_dict_list=metrics_dict_list):
    eval_dict["activation"]=activation
    eval_dict["type"]=type
    eval_dict["start"]=start
    eval_dict["phase"]=phase
    eval_dict["end"]=end

    metrics_dict_list.append(eval_dict)


Testing the above function with a simple 50 hidden layer node model

In [ ]:
print( "t1_"   +str(50)+"_bipolar_sigmoid_train_start \n")
time_log["t1_" +str(50)+"_bipolar_sigmoid_train_start"]=time.time()
t1_models["t1_"+str(50)+"_bipolar_sigmoid"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=50, output_nodes=71, activation=_bipolar_sigmoid)
predicted, eval_dict=t1_models["t1_"+str(50)+"_bipolar_sigmoid"].fit(type1_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
time_log["t1_" +str(50)+"_bipolar_sigmoid_train_end"]=time.time()

add_data_to_metric_list(eval_dict, "_bipolar_sigmoid", "type1_BERT_Embeddings_Train", time_log["t1_" +str(50)+"_bipolar_sigmoid_train_start"], "train", time_log["t1_" +str(50)+"_bipolar_sigmoid_train_end"])

print( "t1_"   +str(50)+"_bipolar_sigmoid_test_start \n")
time_log["t1_" +str(50)+"_bipolar_sigmoid_test_start"]=time.time()
predicted, eval_dict=t1_models["t1_"+str(50)+"_bipolar_sigmoid"].predict(type1_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
time_log["t1_" +str(50)+"_bipolar_sigmoid_test_end"]=time.time()

add_data_to_metric_list(eval_dict, "_bipolar_sigmoid", "type1_BERT_Embeddings_Test", time_log["t1_" +str(50)+"_bipolar_sigmoid_test_start"], "test", time_log["t1_" +str(50)+"_bipolar_sigmoid_test_end"])

metrics_dict_list

TRAINING

In [ ]:
for i in list_of_models_hidden_nodes:

    print( "t1_"   +str(i)+"_bipolar_sigmoid_train_start \n")
    time_log["t1_" +str(i)+"_bipolar_sigmoid_train_start"]=time.time()
    t1_models["t1_"+str(i)+"_bipolar_sigmoid"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=i, output_nodes=71, activation=_bipolar_sigmoid)
    predicted, eval_dict=t1_models["t1_"+str(i)+"_bipolar_sigmoid"].fit(type1_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
    time_log["t1_" +str(i)+"_bipolar_sigmoid_train_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_bipolar_sigmoid", "type1_BERT_Embeddings_Train", time_log["t1_" +str(i)+"_bipolar_sigmoid_train_start"], "train", time_log["t1_" +str(i)+"_bipolar_sigmoid_train_end"])

    
    print( "t1_"   +str(i)+"_relu_leaky_train_start \n")
    time_log["t1_" +str(i)+"_relu_leaky_train_start"]=time.time()
    t1_models["t1_"+str(i)+"_relu_leaky"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=i, output_nodes=71, activation=_relu_leaky)
    predicted, eval_dict=t1_models["t1_"+str(i)+"_relu_leaky"].fit(type1_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
    time_log["t1_" +str(i)+"_relu_leaky_train_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_relu_leaky", "type1_BERT_Embeddings_Train", time_log["t1_" +str(i)+"_relu_leaky_train_start"], "train", time_log["t1_" +str(i)+"_relu_leaky_train_end"])


    print( "t1_"   +str(i)+"_biploar_step_train_start \n")
    time_log["t1_" +str(i)+"_biploar_step_train_start"]=time.time()
    t1_models["t1_"+str(i)+"_biploar_step"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=i, output_nodes=71, activation=_biploar_step)
    predicted, eval_dict=t1_models["t1_"+str(i)+"_biploar_step"].fit(type1_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
    time_log["t1_" +str(i)+"_biploar_step_train_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_biploar_step", "type1_BERT_Embeddings_Train", time_log["t1_" +str(i)+"_biploar_step_train_start"], "train", time_log["t1_" +str(i)+"_biploar_step_train_end"])


    print( "t2_"   +str(i)+"_bipolar_sigmoid_train_start \n")
    time_log["t2_" +str(i)+"_bipolar_sigmoid_train_start"]=time.time()
    t2_models["t2_"+str(i)+"_bipolar_sigmoid"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=i, output_nodes=71, activation=_bipolar_sigmoid)
    predicted, eval_dict=t2_models["t2_"+str(i)+"_bipolar_sigmoid"].fit(type2_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
    time_log["t2_" +str(i)+"_bipolar_sigmoid_train_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_bipolar_sigmoid", "type2_BERT_Embeddings_Train", time_log["t2_" +str(i)+"_bipolar_sigmoid_train_start"], "train", time_log["t2_" +str(i)+"_bipolar_sigmoid_train_end"])


    print( "t2_"   +str(i)+"_relu_leaky_train_start \n")
    time_log["t2_" +str(i)+"_relu_leaky_train_start"]=time.time()
    t2_models["t2_"+str(i)+"_relu_leaky"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=i, output_nodes=71, activation=_relu_leaky)
    predicted, eval_dict=t2_models["t2_"+str(i)+"_relu_leaky"].fit(type2_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
    time_log["t2_" +str(i)+"_relu_leaky_train_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_relu_leaky", "type2_BERT_Embeddings_Train", time_log["t2_" +str(i)+"_relu_leaky_train_start"], "train", time_log["t2_" +str(i)+"_relu_leaky_train_end"])


    print( "t2_"   +str(i)+"_biploar_step_train_start \n")
    time_log["t2_" +str(i)+"_biploar_step_train_start"]=time.time()
    t2_models["t2_"+str(i)+"_biploar_step"]=ELM_MultiLabel(input_nodes=768, hidden_nodes=i, output_nodes=71, activation=_biploar_step)
    predicted, eval_dict=t2_models["t2_"+str(i)+"_biploar_step"].fit(type2_BERT_Embeddings_Train,label_values_Train, verbose=False, show_metrics=True)
    time_log["t2_" +str(i)+"_biploar_step_train_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_biploar_step", "type2_BERT_Embeddings_Train", time_log["t2_" +str(i)+"_biploar_step_train_start"], "train", time_log["t2_" +str(i)+"_biploar_step_train_end"])



TESTING

In [ ]:
for i in list_of_models_hidden_nodes:
    print( "t1_"   +str(i)+"_bipolar_sigmoid_test_start \n")
    time_log["t1_" +str(i)+"_bipolar_sigmoid_test_start"]=time.time()
    predicted, eval_dict=t1_models["t1_"+str(i)+"_bipolar_sigmoid"].predict(type1_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
    time_log["t1_" +str(i)+"_bipolar_sigmoid_test_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_bipolar_sigmoid", "type1_BERT_Embeddings_Test", time_log["t1_" +str(i)+"_bipolar_sigmoid_test_start"], "test", time_log["t1_" +str(i)+"_bipolar_sigmoid_test_end"])


    print( "t1_"   +str(i)+"_relu_leaky_test_start \n")
    time_log["t1_" +str(i)+"_relu_leaky_test_start"]=time.time()
    predicted, eval_dict=t1_models["t1_"+str(i)+"_relu_leaky"].predict(type1_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
    time_log["t1_" +str(i)+"_relu_leaky_test_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_relu_leaky", "type1_BERT_Embeddings_Test", time_log["t1_" +str(i)+"_relu_leaky_test_start"], "test", time_log["t1_" +str(i)+"_relu_leaky_test_end"])


    print( "t1_"   +str(i)+"_biploar_step_test_start \n")
    time_log["t1_" +str(i)+"_biploar_step_test_start"]=time.time()
    predicted, eval_dict=t1_models["t1_"+str(i)+"_biploar_step"].predict(type1_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
    time_log["t1_" +str(i)+"_biploar_step_test_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_biploar_step", "type1_BERT_Embeddings_Test", time_log["t1_" +str(i)+"_biploar_step_test_start"], "test", time_log["t1_" +str(i)+"_biploar_step_test_end"])


    print( "t2_"   +str(i)+"_bipolar_sigmoid_test_start \n")
    time_log["t2_" +str(i)+"_bipolar_sigmoid_test_start"]=time.time()
    predicted, eval_dict=t2_models["t2_"+str(i)+"_bipolar_sigmoid"].predict(type2_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
    time_log["t2_" +str(i)+"_bipolar_sigmoid_test_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_bipolar_sigmoid", "type2_BERT_Embeddings_Test", time_log["t2_" +str(i)+"_bipolar_sigmoid_test_start"], "test", time_log["t2_" +str(i)+"_bipolar_sigmoid_test_end"])


    print( "t2_"   +str(i)+"_relu_leaky_test_start \n")
    time_log["t2_" +str(i)+"_relu_leaky_test_start"]=time.time()
    predicted, eval_dict=t2_models["t2_"+str(i)+"_relu_leaky"].predict(type2_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
    time_log["t2_" +str(i)+"_relu_leaky_test_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_relu_leaky", "type2_BERT_Embeddings_Test", time_log["t2_" +str(i)+"_relu_leaky_test_start"], "test", time_log["t2_" +str(i)+"_relu_leaky_test_end"])

    print( "t2_"   +str(i)+"_biploar_step_test_start \n")
    time_log["t2_" +str(i)+"_biploar_step_test_start"]=time.time()
    predicted, eval_dict=t2_models["t2_"+str(i)+"_biploar_step"].predict(type2_BERT_Embeddings_Test,label_values_Test, show_metrics=True)
    time_log["t2_" +str(i)+"_biploar_step_test_end"]=time.time()

    add_data_to_metric_list(eval_dict, "_biploar_step", "type2_BERT_Embeddings_Test", time_log["t2_" +str(i)+"_biploar_step_test_start"], "test", time_log["t2_" +str(i)+"_biploar_step_test_end"])


In [ ]:
Final_metrics_df= pd.DataFrame(metrics_dict_list)

In [ ]:
Final_metrics_df.to_csv("Final_ELM_Mertics_BERT.csv")

In [ ]:
Final_metrics_df

Doing Testing